### 1. Import and Install Dependencies


In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time 
import mediapipe as mp

### 2. Keypoints using MP Holistic


In [2]:
mp_holistic = mp.solutions.holistic # holistics model
mp_drawing = mp.solutions.drawing_utils  # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert image from BGR to RBG
    image.flags.writeable = False # image is no longer  writable
    result = model.process(image) # make prdictions
    image.flags.writeable = True # image is now writable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # convert image from RBG tto BGR
    return image, result

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw Face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)  # Draw Pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw LEFT hand connection
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw RIGHT hand connections
    

In [5]:
def draw_style_landmarks(image, results):
    # Draw Face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, 
                              mp_holistic.FACEMESH_CONTOURS, 
                              mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)
                             ) 
    
    # Draw Pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, 
                              mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10),thickness=2,circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2)
                             )  
    
     # Draw LEFT hand connection
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, 
                              mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2)
                             )
    
    # Draw RIGHT hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, 
                              mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66,230),thickness=1,circle_radius=1)
                             ) 
    

In [ ]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: 
    while cap.isOpened():
        
        # read frames of each immge video
        ret, frame = cap.read()

        # Make predictions
        image, results = mediapipe_detection(frame, holistic)
        
        
        # draw landmarks on video frame
        draw_style_landmarks(image, results)
        
        # show the frame to the user
        cv2.imshow('Sign Language feeds', image)

        # beak from the frame or quit the shown frame to the user
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break   
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
draw_style_landmarks(image, results)

In [ ]:
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

### 3. Extract Keypoint Values

In [ ]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmarks]).flatten() if results.pose_landmarks.landmarks else np.zeros(132)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmarks]).flatten() if results.left_hand_landmarks.landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmarks]).flatten() if results.right_hand_landmarks.landmarks else np.zeros(21*3)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmarks]).flatten() if results.face_landmarks.landmarks else np.zeros(1404) 
   

In [49]:
np.zeros(21*3)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [52]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmarks]).flatten() if results.pose_landmarks.landmarks else np.zeros(132)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmarks]).flatten() if results.left_hand_landmarks.landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmarks]).flatten() if results.right_hand_landmarks.landmarks else np.zeros(21*3)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmarks]).flatten() if results.face_landmarks.landmarks else np.zeros(1404) 
    return np.concatenate(pose, face, lh, rh)

### 4. Setup Folders for Collection

In [21]:
# path to get exported data numpy array
DATA_PATH = os.path.join('MP_DATA')

# list of actions to be etecte as our label
actions = np.array(['hello','thanks', 'iloveyou'])

# thirty videos worth of data
no_sequence = 30

# video has a length of thirty frames
sequence_length = 30


In [29]:
for action in actions:
    for sequence in range(no_sequence):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

### 5. Collect Keypoints Value for Training and Testing 

### 6. Process Data and Create labels and features

### 7. Build and Train LSTM Neural Network

### 8. Make Predictions

### 9. Save Weights

### 10. Evalute using Confusion Matrix and Accuracy

### 11. Text in Real Time